In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 4

In [2]:
pip install datasets

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from datasets import load_dataset
import os
from sklearn.metrics import classification_report


from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [ ]:
#load the data for all languages
data = load_dataset("Eloquent/HalluciGen-Translation", "test_detection")

print(data.keys())

data_de_en = data['test_detection_de_en'].to_pandas()
data_en_de = data['test_detection_en_de'].to_pandas()
data_fr_en = data['test_detection_fr_en'].to_pandas()
data_en_fr = data['test_detection_en_fr'].to_pandas()

display(data_de_en.head())
display(data_en_de.head())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating test_detection_de_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_de split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_fr_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_detection_en_fr split:   0%|          | 0/100 [00:00<?, ? examples/s]

dict_keys(['test_detection_de_en', 'test_detection_en_de', 'test_detection_fr_en', 'test_detection_en_fr'])


,id,langpair,source,hyp1,hyp2
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In developed countries, offering luxury guesth..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice elsewhere in the UK, b..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"“Based on this fossil, we can say that the sep...","“Based on this fossil, we can say that the sep..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never overtake a car, build a well in ...","We will never overtake a car, build a well in ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","An Avenger class anti-mining ship, the ship wa...","An Avenger class anti-mining ship, the ship wa..."


,id,langpair,source,hyp1,hyp2
0,0,en-de,He graduated from the College of Arts & Scienc...,Er absolvierte 1950 das College of Arts & Scie...,Er absolvierte 1950 das College of Arts & Scie...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten der kleineren Inseln sind unabhäng...,"Die meisten sind unabhängige Nationen, oder mi..."
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die immer...","Aber es gibt viele Dinge über Vögel, die wie e..."
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...",Die neuseeländische Polizei hatte Schwierigkei...
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","""Das Plateau von Giza oder """"Nekropolis von Gi...","""Das Plateau von Giza oder """"Nekropolis von Gi..."


In [4]:

#load the trial data for all language pairs
trial_ds = load_dataset("Eloquent/HalluciGen-Translation", name="trial")
print(trial_ds.keys())

Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating trial_de_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_de split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_fr_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_fr split:   0%|          | 0/10 [00:00<?, ? examples/s]

dict_keys(['trial_de_en', 'trial_en_de', 'trial_fr_en', 'trial_en_fr'])


In [5]:


trial_ds_de_en = trial_ds['trial_de_en'].to_pandas()
trial_ds_en_de = trial_ds['trial_en_de'].to_pandas()
trial_ds_fr_en = trial_ds['trial_fr_en'].to_pandas()
trial_ds_en_fr = trial_ds['trial_en_fr'].to_pandas()

display(trial_ds_de_en.head())
display(trial_ds_en_de.head())



,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2


,id,langpair,source,hyp1,hyp2,type,label
0,0,en-de,The days in the summer can lead to problems ge...,"Die Tage im Sommer können zu Problemen führen,...",Die sehr langen Tage im Sommer können zu Probl...,addition,hyp2
1,1,en-de,The vertical clearance under the bridge is 15 ...,Der vertikale Abstand unter der Brücke beträgt...,Der vertikale Abstand unter der Brücke beträgt...,date,hyp1
2,2,en-de,William Llewelyn Williams known as Llewelyn Wi...,"William Williams, bekannt als Llewelyn William...","Williajm Llewelyn Williams, bekannt als Llewel...",named-entity,hyp2
3,3,en-de,The single was released on Radio Airplay in It...,Die Single wurde am 12. Oktober 212 in Italien...,Die Single wurde am 12. Oktober 2012 in Italie...,number,hyp1
4,4,en-de,Water is spilling over the levee in a section ...,Wasser fließt über den Deich in einem Abschnit...,Wasser fließt über die Leibe in einem Abschnit...,natural,hyp2


In [6]:
trial_ds_de_en

,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2


## Load Model

In [ ]:
#load model Llama3
model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt

    answer_format = {"label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is hallucinated ("label").
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

### Test

In [ ]:
source = trial_ds_de_en.iloc[0]["source"]
hyp1 = trial_ds_de_en.iloc[0]["hyp1"]
hyp2 = trial_ds_de_en.iloc[0]["hyp2"]
hallucination = detect_hallucination_prompt1(source, hyp1, hyp2, "English")
json_data = extract_json_data(hallucination)
print(json_data)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label").
      Provide the result in the following format: {'label': ''}.

       Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
       hyp1 : In the phase, wandering ants march at night and rest during the day.
       hyp2 : In the nomadic phase, wandering ants march at night and rest during the day.

       Result:
       
detected hallucination:  {'label': 'hyp2'}
{'label': 'hyp2'}


In [ ]:
trial_ds_de_en.iloc[0]["label"]


'hyp2'

# Test with Trial dataset
We test different prompts to see which prompt leads to the best results on the trial dataset.

### Prompt 1

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label").
Provide the result in the following format: {"label": ""}.

Src: {source}


hyp1 : {hyp1}


hyp2 : {hyp2}


Result:


In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallucination(source, hyp1, hyp2, translation_language)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]

                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en.csv"
skipped_rows = save_trial_results(trial_ds_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'hyp1'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

detected hallucination:  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


detected hallucination:  {'label': 'hyp2'}
detected hallucination:  {'label': 'hyp2'}


In [ ]:
df = pd.read_csv(csv_filename)

display(df)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en.txt"

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['pred_label'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)


,id,langpair,source,hyp1,hyp2,type,label,pred_label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp2
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.20      0.33         5
        hyp2       0.56      1.00      0.71         5

    accuracy                           0.60        10
   macro avg       0.78      0.60      0.52        10
weighted avg       0.78      0.60      0.52        10



## Prompt 2

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
Provide the result in the following format: {"type": "", "label": ""}.


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
#save results
def save_trial_results(df, csv_filename, translation_language):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallucination(source, hyp1, hyp2, translation_language)
                extracted_hallu = extract_json_data(result)
                hallu_label = extracted_hallu["label"]
                hallu_type = extracted_hallu["type"]


                # Update DataFrame with new values
                chunk.at[index, "pred_label"] = hallu_label
                chunk.at[index, "pred_type"] = hallu_type

            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows, df

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt2.csv"
skipped_rows = save_trial_results(trial_ds_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n       hyp1 : In the phase, wandering ants march at night and rest during the day.\n       hyp2 : In the nomadic phase, wandering ants march at night and rest during the day.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{\'type\':\'specialized vocabulary\', \'label\': \'hyp2\'}\n\nThe hallucination in this case is in the phrase "nomadic phase" which is not present in the original source text. This is 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz von der Bühne verletzte, haben sie die Tour abgebrochen.\n       hyp1 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour.\n       hyp2 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on Aug. 5, they canceled the tour.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_heade

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 65 km (40 Meilen) südlich von Butte.\n       hyp1 : The center of the earthquake was about 20 km northeast of Dillon and about 65 metres south of Butte.\n       hyp2 : The center of the earthquake was about 20 km northeast of Dillon and about 65000 metres south of Butte.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{\'type\': \'numerical\', \'label\': \'hyp2\'}\n\nExplanation: \nT

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Boten eingesetzt wurden.\n       hyp1 : There were also 58 liaison aircraft but 20 of these were only used for messengers.\n       hyp2 : There were also 58 liaison aircraft but 20 of these were not only used for messengers.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{\'type\': \'negation\', \'label\': \'hyp2\'}'}]
detected hallucination:  {'type': 'negation', 'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Natriumoxid als Flachglas und einen geringeren Gehalt an Kieselsäure, Calciumoxid und Aluminiumoxid.\n       hyp1 : Container glass has a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.\n       hyp2 : Container glass had a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.\n\n       Result:

In [ ]:
df = pd.read_csv(csv_filename)

display(df)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en_prompt2.txt"

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['pred_label'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)


,id,langpair,source,hyp1,hyp2,type,label,pred_label,pred_type
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,hyp2,specialized vocabulary
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp2,date
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2,gender
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2,temporal hallucination
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2,numeric hallucination
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp2,minor
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2,numerical
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2,negation
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2,none
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2,gender


              precision    recall  f1-score   support

        hyp1       0.00      0.00      0.00         5
        hyp2       0.50      1.00      0.67         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pd.set_option('display.max_colwidth', None)

trial_ds_de_en.head(10)

,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.,"In the phase, wandering ants march at night and rest during the day.","In the nomadic phase, wandering ants march at night and rest during the day.",addition,hyp2
1,1,de-en,"Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz von der Bühne verletzte, haben sie die Tour abgebrochen.","After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour.","After front singer Steven Tyler injured himself in a fall from the stage during a performance on Aug. 5, they canceled the tour.",date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin beim Schummeln erwischt.,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch Olusegun Obasanjo (1975) und Murtala Mohammed (1976) ersetzt.,He was replaced in successive coups by Olusegun Obasanjo (1975) and Murtalea Mohammed (1976).,He was replaced by Olusegun Obasanjo (in 1975) and Murtala Mohammed (in 1976) in successive coups.,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Dezember 2017 offiziell angekündigt.,"On December 7, 2017 NVIDIA officially announced the Nvidia TITAN V.","The NVIDIA TITAN V was officially announced by Nvidia on December 7, 92017.",number,hyp2
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde kämpft. Da der Sattel noch nicht erfunden war, kämpfte die assyrische Kavallerie vom bloßen Rücken ihrer Pferde aus.","A cavalry is a troop that fights on horses.Since the seat was not yet invented, the Assyrian cavalry fought from the bare back of their horses.","A cavalry is a troop that fights on horses.Since the saddle was not yet invented, the Assyrian cavalry fought from the bare back of their horses.",natural,hyp1
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 65 km (40 Meilen) südlich von Butte.,The center of the earthquake was about 20 km northeast of Dillon and about 65 metres south of Butte.,The center of the earthquake was about 20 km northeast of Dillon and about 65000 metres south of Butte.,conversion,hyp1
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Boten eingesetzt wurden.",There were also 58 liaison aircraft but 20 of these were only used for messengers.,There were also 58 liaison aircraft but 20 of these were not only used for messengers.,negation,hyp2
8,8,de-en,"Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Natriumoxid als Flachglas und einen geringeren Gehalt an Kieselsäure, Calciumoxid und Aluminiumoxid.","Container glass has a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.","Container glass had a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.",tense,hyp2
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) sind finanziell gut aufgestellt.,Kathy and her husband Peter Dean (Pete Beale) are financially stable.,Kathy and her wife Pete Beale (Peter Dean) are stable financially.,antonym,hyp2


## Prompt 3

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
Provide the result in the following format: {answer_format}. Here is an example to illustrate this:



Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.


hyp1: In the phase, wandering ants march at night and rest during the day.


hyp2: In the nomadic phase, wandering ants march at night and rest during the day.


label: hyp2


type: addition



Detect which of the following hypotheses is the hallucination.

Src: {source}

hyp1 : {hyp1}

hyp2 : {hyp2}

Result:
      

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt3.csv"
skipped_rows, df = save_trial_results(trial_ds_de_en.iloc[1:], csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz vo

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Der Wirtschaftsprüfer hat die Entwicklerin beim Schummeln erwischt.\n       hyp1 : The auditor cau

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 6

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Boten eingesetzt wurden.\n       hyp1 : 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Natriumoxid als Flachglas und einen ger

In [ ]:

display(df)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en_prompt3.txt"

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['pred_label'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)


,id,langpair,source,hyp1,hyp2,type,label,pred_label,pred_type
0,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1,none
1,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2,modification
2,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2,specification
3,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2,modification
4,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp1,no hallucination
5,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2,magnitude
6,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2,negation
7,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2,tense
8,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2,substitution


              precision    recall  f1-score   support

        hyp1       1.00      0.40      0.57         5
        hyp2       0.57      1.00      0.73         4

    accuracy                           0.67         9
   macro avg       0.79      0.70      0.65         9
weighted avg       0.81      0.67      0.64         9



## Prompt 4

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
  

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt4.csv"
skipped_rows, df = save_trial_results(trial_ds_de_en.iloc[1:], csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is. \n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Nachdem sich der Frontsänger Steven Tyler während eines Auftri

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is. \n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Der Wirtschaftsprüfer hat die Entwicklerin beim Schummeln erwi

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is. \n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) n

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is. \n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Bote

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is. \n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Nat

In [ ]:
df.head()

,id,langpair,source,hyp1,hyp2,type,label,pred_label,pred_type
0,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1,none
1,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2,substitution
2,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2,addition
3,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2,error
4,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp1,none


In [ ]:
df_without_nan = df.dropna()

# Anzeige des DataFrames ohne NaN-Werte
display(df_without_nan)

# Annahme: classification_report und classification_report_file sind definiert
# Erstelle den Klassifikationsbericht
report = classification_report(df_without_nan['label'], df_without_nan['pred_label'])

print(report)

,id,langpair,source,hyp1,hyp2,type,label,pred_label,pred_type
0,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1,none
1,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2,substitution
2,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2,addition
3,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2,error
4,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp1,none
6,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2,addition
7,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2,tense
8,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2,addition


              precision    recall  f1-score   support

        hyp1       1.00      0.50      0.67         4
        hyp2       0.67      1.00      0.80         4

    accuracy                           0.75         8
   macro avg       0.83      0.75      0.73         8
weighted avg       0.83      0.75      0.73         8



## Prompt 5

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.
      Explain your choice.
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.
      Explain your choice.
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt5.csv"
skipped_rows, df = save_trial_results(trial_ds_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n       hyp1 : In the phase, wandering ants march at night and rest during the day.\n       hyp2 : In the nomadic phase, wandering ants march at night and rest during the day.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nBased on the given source and hypotheses, I detect that hypothesis "hyp2" is not supported by the source and determine that it is a "type" of

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz von der Bühne verletzte, haben sie die Tour abgebrochen.\n       hyp1 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour.\n       hyp2 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on Aug. 5, they canceled the tour.\n\n       Result:<|eot_

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 65 km (40 Meilen) südlich von Butte.\n       hyp1 : The center of the earthquake was about 20 km northeast of Dillon and about 65 metres south of Butte.\n       hyp2 : The center of the earthquake was about 20 km northeast of Dillon and about 65000 metres south of Butte.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAfter analyzing the sou

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Boten eingesetzt wurden.\n       hyp1 : There were also 58 liaison aircraft but 20 of these were only used for messengers.\n       hyp2 : There were also 58 liaison aircraft but 20 of these were not only used for messengers.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAfter analyzing the source text and the two hypotheses, I detect that hypothesis 2 is not supported by the so

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Natriumoxid als Flachglas und einen geringeren Gehalt an Kieselsäure, Calciumoxid und Aluminiumoxid.\n       hyp1 : Container glass has a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.\n       hyp2 : Container glass had a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium 

In [ ]:
df_without_nan = df.dropna()

# Anzeige des DataFrames ohne NaN-Werte
display(df_without_nan)

# Annahme: classification_report und classification_report_file sind definiert
# Erstelle den Klassifikationsbericht
report = classification_report(df_without_nan['label'], df_without_nan['pred_label'])

print(report)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en_prompt5.txt"

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)

,id,langpair,source,hyp1,hyp2,type,label,pred_label,pred_type
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,hyp2,Lexical
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1,inconsistent information
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2,attribute
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2,Extra information
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2,numeric
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp2,typical
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2,magnitude
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2,Temporal Hallucination
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2,gender hallucination


              precision    recall  f1-score   support

        hyp1       1.00      0.20      0.33         5
        hyp2       0.50      1.00      0.67         4

    accuracy                           0.56         9
   macro avg       0.75      0.60      0.50         9
weighted avg       0.78      0.56      0.48         9



## Prompt 6

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.
      Explain your choice.
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.
      Explain your choice.
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier and translator. You speak only German and English."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt6.csv"
skipped_rows, df = save_trial_results(trial_ds_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n       hyp1 : In the phase, wandering ants march at night and rest during the day.\n       hyp2 : In the nomadic phase, wandering ants march at night and rest during the day.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAfter analyzing the source text "In der Phase marschieren Wanderameisen bei Nacht und mache

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz von der Bühne verletzte, haben sie die Tour abgebrochen.\n       hyp1 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour.\n       hyp2 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on Aug. 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 65 km (40 Meilen) südlich von Butte.\n       hyp1 : The center of the earthquake was about 20 km northeast of Dillon and about 65 metres south of Butte.\n       hyp2 : The center of the earthquake was about 20 km northeast of Dillon and about 65000 metres south of Butte.\n\n       Result:<|eot_id|><|start_header_id|>ass

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Boten eingesetzt wurden.\n       hyp1 : There were also 58 liaison aircraft but 20 of these were only used for messengers.\n       hyp2 : There were also 58 liaison aircraft but 20 of these were not only used for messengers.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAfter analyzing the source text and the two hypotheses, I c

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported ("label") by the source and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Natriumoxid als Flachglas und einen geringeren Gehalt an Kieselsäure, Calciumoxid und Aluminiumoxid.\n       hyp1 : Container glass has a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.\n       hyp2 : Container glass had a higher content of magnesium oxide and sodium oxide as 

In [ ]:
df_without_nan = df.dropna()

# Anzeige des DataFrames ohne NaN-Werte
display(df_without_nan)

# Annahme: classification_report und classification_report_file sind definiert
# Erstelle den Klassifikationsbericht
report = classification_report(df_without_nan['label'], df_without_nan['pred_label'])

print(report)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en_prompt6.txt"

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)

,id,langpair,source,hyp1,hyp2,type,label,pred_label,pred_type
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,hyp2,extra information
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1,temporal hallucination
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2,attribute
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,hyp2,Omission
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,not supported,date hallucination
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,hyp2,semantic
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2,hallucination
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2,additional information
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2,temporal hallucination
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2,Gender hallucination


               precision    recall  f1-score   support

         hyp1       1.00      0.20      0.33         5
         hyp2       0.50      0.80      0.62         5
not supported       0.00      0.00      0.00         0

     accuracy                           0.50        10
    macro avg       0.50      0.33      0.32        10
 weighted avg       0.75      0.50      0.47        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Prompt 7

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what type of hallucination ("type") it is.
      Explain your choice.
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is not supported by the source and not directly translated
    and determine what type of hallucination ("type") it is.
      Explain your choice.
      Provide the result in the following format: {answer_format}.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier and translator. You speak only German and English."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt7.csv"
skipped_rows, df = save_trial_results(trial_ds_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported and by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n       hyp1 : In the phase, wandering ants march at night and rest during the day.\n       hyp2 : In the nomadic phase, wandering ants march at night and rest during the day.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAfter analyzing the source and the two hypotheses, I detect that "hyp

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported and by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz von der Bühne verletzte, haben sie die Tour abgebrochen.\n       hyp1 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour.\n       hyp2 : After front singer Steven Tyler injured himself in a fall from the stage during 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported and by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 65 km (40 Meilen) südlich von Butte.\n       hyp1 : The center of the earthquake was about 20 km northeast of Dillon and about 65 metres south of Butte.\n       hyp2 : The center of the earthquake was about 20 km northeast of Dillon and about 65000 metres south of Butte.\n\n       Result:<|eot_id|>

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported and by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Boten eingesetzt wurden.\n       hyp1 : There were also 58 liaison aircraft but 20 of these were only used for messengers.\n       hyp2 : There were also 58 liaison aircraft but 20 of these were not only used for messengers.\n\n       Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAfter analyzing the source text and t

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported and by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n       Src: Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Natriumoxid als Flachglas und einen geringeren Gehalt an Kieselsäure, Calciumoxid und Aluminiumoxid.\n       hyp1 : Container glass has a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.\n       hyp2 : Container glass had a higher content of magnesium oxid

In [ ]:

report = classification_report(df['label'], df['pred_label'])

print(report)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en_prompt7.txt"

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)

              precision    recall  f1-score   support

        hyp1       1.00      0.40      0.57         5
        hyp2       0.62      1.00      0.77         5

    accuracy                           0.70        10
   macro avg       0.81      0.70      0.67        10
weighted avg       0.81      0.70      0.67        10



## Prompt 8

Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what type of hallucination ("type") it is.
Provide the result in the following format: {answer_format}.
Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is not supported by the source and explain your choice.


       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}.
    Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what
    type of hallucination ("type") it is.
      Provide the result in the following format: {answer_format}.

      Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is not supported by the source and explain your choice.


       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier and translator. You speak only German and English."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt8.csv"
skipped_rows, df = save_trial_results(trial_ds_de_en.iloc[1:], csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n      Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is not supported by the source and explain 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n      Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is not supported by the source and explain 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n      Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is not supported by the source and explain 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n      Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is not supported by the source and explain 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is not supported by the source and not directly translated and determine what type of hallucination ("type") it is.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}.\n\n      Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is not supported by the source and explain 

In [ ]:

report = classification_report(df['label'], df['pred_label'])

print(report)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en_prompt8.txt"

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)

              precision    recall  f1-score   support

        hyp1       1.00      0.40      0.57         5
        hyp2       0.57      1.00      0.73         4

    accuracy                           0.67         9
   macro avg       0.79      0.70      0.65         9
weighted avg       0.81      0.67      0.64         9



## Prompt 9

Given a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to {translation_language}. Then, compare these translations with "src" to detect which of the two hypotheses is not supported by 'src' and what type of hallucination ("type") it is.
    Provide the result in the following format: {answer_format}. YOU HAVE TO CHOOSE BETWEEN "hyp1" AND "hyp2". I DO NOT ACCEPT NOTHING AS "label"!!

    Src: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Result:
    

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to {translation_language}.
    Then, compare these translations with "src" to detect which of the two hypotheses is not supported by 'src' and what type of hallucination ("type") it is.
    Provide the result in the following format: {answer_format}. YOU HAVE TO CHOOSE BETWEEN "hyp1" AND "hyp2". I DO NOT ACCEPT NOTHING AS "label"!!

    Src: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Result:
    '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier and translator. You speak only German and English."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_results_llama3_de_en_prompt9.csv"
skipped_rows, df = save_trial_results(trial_ds_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to English. Then, compare these translations with "src" to detect which of the two hypotheses is not supported by \'src\' and what type of hallucination ("type") it is.\n    Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. YOU HAVE TO CHOOSE BETWEEN "hyp1" AND "hyp2". I DO NOT ACCEPT NOTHING AS "label"!!\n\n    Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n    hyp1 : In the phase, wandering ants march at night and rest during the day.\n    hyp2 : In the nomadic phase, wandering ants march at night and rest during the day.\n\n    Result:<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI\'ll tran

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to English. Then, compare these translations with "src" to detect which of the two hypotheses is not supported by \'src\' and what type of hallucination ("type") it is.\n    Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. YOU HAVE TO CHOOSE BETWEEN "hyp1" AND "hyp2". I DO NOT ACCEPT NOTHING AS "label"!!\n\n    Src: Nachdem sich der Frontsänger Steven Tyler während eines Auftrittes am 5. August bei einem Sturz von der Bühne verletzte, haben sie die Tour abgebrochen.\n    hyp1 : After front singer Steven Tyler injured himself in a fall from the stage during a performance on December 5, they canceled the tour.\n    hyp2 : After front singer

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to English. Then, compare these translations with "src" to detect which of the two hypotheses is not supported by \'src\' and what type of hallucination ("type") it is.\n    Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. YOU HAVE TO CHOOSE BETWEEN "hyp1" AND "hyp2". I DO NOT ACCEPT NOTHING AS "label"!!\n\n    Src: Das Zentrum des Erdbebens befand sich etwa 20 km (15 Meilen) nordnordöstlich von Dillon und etwa 65 km (40 Meilen) südlich von Butte.\n    hyp1 : The center of the earthquake was about 20 km northeast of Dillon and about 65 metres south of Butte.\n    hyp2 : The center of the earthquake was about 20 km northeast of Dillon and 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to English. Then, compare these translations with "src" to detect which of the two hypotheses is not supported by \'src\' and what type of hallucination ("type") it is.\n    Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. YOU HAVE TO CHOOSE BETWEEN "hyp1" AND "hyp2". I DO NOT ACCEPT NOTHING AS "label"!!\n\n    Src: Es gab auch 58 Verbindungsflugzeuge, von denen 20 nur für Boten eingesetzt wurden.\n    hyp1 : There were also 58 liaison aircraft but 20 of these were only used for messengers.\n    hyp2 : There were also 58 liaison aircraft but 20 of these were not only used for messengers.\n\n    Result:<|eot_id|><|start_header_id|>assistan

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier and translator. You speak only German and English.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", your task is to first translate "hyp1" and "hyp2" back to English. Then, compare these translations with "src" to detect which of the two hypotheses is not supported by \'src\' and what type of hallucination ("type") it is.\n    Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. YOU HAVE TO CHOOSE BETWEEN "hyp1" AND "hyp2". I DO NOT ACCEPT NOTHING AS "label"!!\n\n    Src: Behälterglas hat einen höheren Gehalt an Magnesiumoxid und Natriumoxid als Flachglas und einen geringeren Gehalt an Kieselsäure, Calciumoxid und Aluminiumoxid.\n    hyp1 : Container glass has a higher content of magnesium oxide and sodium oxide as flat glass and a lower content of silica, calcium oxide and aluminum oxide.\n   

In [ ]:
df.iloc[:10]

,id,langpair,source,hyp1,hyp2,type,label,pred_label,pred_type
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2,hyp1,omission
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1,hyp1,date_format
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1,hyp2,Gender-specific information
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1,NaN,NaN
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2,hyp2,date hallucination
5,5,de-en,"Eine Kavallerie ist eine Truppe, die zu Pferde...",A cavalry is a troop that fights on horses.Sin...,A cavalry is a troop that fights on horses.Sin...,natural,hyp1,NaN,NaN
6,6,de-en,Das Zentrum des Erdbebens befand sich etwa 20 ...,The center of the earthquake was about 20 km n...,The center of the earthquake was about 20 km n...,conversion,hyp1,hyp2,Numerical Hallucination
7,7,de-en,"Es gab auch 58 Verbindungsflugzeuge, von denen...",There were also 58 liaison aircraft but 20 of ...,There were also 58 liaison aircraft but 20 of ...,negation,hyp2,hyp2,addition
8,8,de-en,Behälterglas hat einen höheren Gehalt an Magne...,Container glass has a higher content of magnes...,Container glass had a higher content of magnes...,tense,hyp2,hyp2,past tense hallucination
9,9,de-en,Kathy und ihr Ehemann Pete Beale (Peter Dean) ...,Kathy and her husband Peter Dean (Pete Beale) ...,Kathy and her wife Pete Beale (Peter Dean) are...,antonym,hyp2,hyp2,gender hallucination


In [ ]:
df_without_nan = df.dropna()


# Erstelle den Klassifikationsbericht
report = classification_report(df_without_nan['label'], df_without_nan['pred_label'])

print(report)

classification_report_file = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/trial_report_llama3_de_en_prompt9.txt"

# Speichere den Klassifikationsbericht in eine Textdatei
with open(classification_report_file, "w") as text_file:
    text_file.write(report)

              precision    recall  f1-score   support

        hyp1       0.50      0.33      0.40         3
        hyp2       0.67      0.80      0.73         5

    accuracy                           0.62         8
   macro avg       0.58      0.57      0.56         8
weighted avg       0.60      0.62      0.60         8



## Generate dataset
The best result were achieved using prompt 4, so that is the prompt we are going for.

In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
      hyp1: In the phase, wandering ants march at night and rest during the day.
      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_de_en_prompt4.csv"
skipped_rows, df = save_trial_results(data_de_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: In Ländern ist das Anbieten von Luxus-Gästehäusern heute zu ein

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Dies ist in Großbritannien gängige Praxis, aber die schottische

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Der Ponoru ist eine Nebenfluss des Horezu in Rumänien.\n       

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Leslie Waddington eröffnete 1966 seine eigene Galerie in der Co

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.\n      hyp1: In the phase, wandering ants march at night and rest during the day.\n      hyp2: In the nomadic phase, wandering ants march at night and rest during the day.\n      label: hyp2\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: In der Zwischenzeit kaufte die Air Force die SR71, nun A-12, an

#English-German

In [ ]:
trial_ds_en_de.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,en-de,The days in the summer can lead to problems getting sufficient sleep and associated health issues.,"Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.","Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.",addition,hyp2
1,1,en-de,"The vertical clearance under the bridge is 15 meters. Construction was completed in August 2011, it didn't open to traffic until March 2017.","Der vertikale Abstand unter der Brücke beträgt 15 Meter.Der Bau wurde im August 2011 abgeschlossen, er wurde erst im Januar 2017 für den Verkehr geöffnet.","Der vertikale Abstand unter der Brücke beträgt 15 Meter.Der Bau wurde im Aug. 2011 abgeschlossen, er wurde erst im März 2017 für den Verkehr geöffnet.",date,hyp1
2,2,en-de,"William Llewelyn Williams known as Llewelyn Williams (10 March 1867 -- 22 April 1922), was a Welsh journalist, lawyer and radical Liberal Party politician.","William Williams, bekannt als Llewelyn Williams (10. März 1867-22. April 1922) war ein walisischer Journalist, Anwalt und radikaler Politiker der Liberalen Partei.","Williajm Llewelyn Williams, bekannt als Llewelyn Williams (10. März 1867-22. April 1922), war ein walisischer Journalist, Anwalt und radikaler Politiker der Liberalen Partei.",named-entity,hyp2
3,3,en-de,"The single was released on Radio Airplay in Italy on October 12, 2012, and was shipped worldwide on 3 December 2012.",Die Single wurde am 12. Oktober 212 in Italien zum Radio Airplay veröffentlicht und am 3. Dezember 2012 weltweit verschickt.,Die Single wurde am 12. Oktober 2012 in Italien zum Radio Airplay veröffentlicht und am 3. Dezember 2012 weltweit verschickt.,number,hyp1
4,4,en-de,Water is spilling over the levee in a section 100 feet wide.,Wasser fließt über den Deich in einem Abschnitt von 100 Fuß breit.,Wasser fließt über die Leibe in einem Abschnitt von 100 Fuß breit.,natural,hyp2


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.
      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.
      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.
      label: hyp2
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_de_prompt4.csv"
skipped_rows, df = save_trial_results(data_en_de, csv_filename, "German")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into German. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.\n      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      label: hyp2\n      type: addition\n\n      Detect which of th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into German. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.\n      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      label: hyp2\n      type: addition\n\n      Detect which of th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into German. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.\n      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      label: hyp2\n      type: addition\n\n      Detect which of th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into German. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.\n      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      label: hyp2\n      type: addition\n\n      Detect which of th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into German. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The days in the summer can lead to problems getting sufficient sleep and associated health issues.\n      hyp1: Die Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      hyp2: Die sehr langen Tage im Sommer können zu Problemen führen, genügend Schlaf zu bekommen und damit verbundene Gesundheitsprobleme.\n      label: hyp2\n      type: addition\n\n      Detect which of th

In [ ]:
skipped_rows

[19, 29, 54]

# English-French

In [ ]:
pd.set_option('display.max_colwidth', None)
trial_ds_en_fr.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,en-fr,The Madhya Police recovered the stolen laptop and mobile phone.,La police de Madhya Pradesh a récupéré l'ordinateur portable et le téléphone portable volés.,La police de Madhya a récupéré l'ordinateur portable et le téléphone portable volés.,addition,hyp1
1,1,en-fr,The vote must still be ratified by the full IOC at it's October meeting in Copenhagen.,Le vote doit encore être ratifié par le CIO lors de sa réunion d’oct. à Copenhague.,Le vote doit encore être ratifié par le CIO lors de sa réunion d’avril à Copenhague.,date,hyp2
2,2,en-fr,"The Mumbai attackers arrived via boat on Novemeber 26, 2008, bringing with them grenades, automatic weapons and hit multiple targets including the crowded Chhatrapati Shivaji Terminus railway station and the famous Taj Mahal Hotel.","Les assaillants de Mumbai sont arrivés par bateau le 26 juillet 2008, apportant avec eux des grenades, des armes automatiques et frappant plusieurs cibles, notamment la gare de Chhatrapati Shivaji Terminus et le célèbre Taj Mahal Hotel.","Les assaillants de Mumbai sont arrivés par bateau le 26 nov. 2008, apportant avec eux des grenades, des armes automatiques et frappant plusieurs cibles, notamment la gare de Chhatrapati Shivaji Terminus et le célèbre Taj Mahal Hotel.",date,hyp1
3,3,en-fr,"Among the prominent women of Suriname are Jennifer Simons, Marijke Djwalapersad, Elisabeth Samson, Cynthia McLeod, and Ruth Wijdenbosch.","Parmi les femmes éminentes du Suriname figurent Jennifer Simons, Marijke Djwalapersad, Elisabeth Samson, Cynthia McLeod et Ruth Wijdenbosch.","Jennifer Simons, Marijke Djwalapersad, Elisabeth Samson, Cynthia McLeol et Ruth Wijdenbosch comptent parmi les femmes influentes du Suriname.",named-entity,hyp2
4,4,en-fr,Taylor remained active until his death as a scout for the Chicago White Sox and the Milwaukee and Atlanta Braves in the baseball.,Taylopr est resté actif dans le baseball en tant que recruteur pour les Atlanta Braves ainsi que les Milwaukee et Chicago White Sox jusqu'à sa mort.,Taylor est resté actif dans le baseball en tant que recruteur pour les Atlanta Braves ainsi que les Milwaukee et Chicago White Sox jusqu'à sa mort.,named-entity,hyp1


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: The Madhya Police recovered the stolen laptop and mobile phone.
      hyp1: La police de Madhya Pradesh a récupéré l'ordinateur portable et le téléphone portable volés.
      hyp2: La police de Madhya a récupéré l'ordinateur portable et le téléphone portable volés.
      label: hyp1
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_fr_prompt4.csv"
skipped_rows, df = save_trial_results(data_en_fr, csv_filename, "French")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into French. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The Madhya Police recovered the stolen laptop and mobile phone.\n      hyp1: La police de Madhya Pradesh a récupéré l\'ordinateur portable et le téléphone portable volés.\n      hyp2: La police de Madhya a récupéré l\'ordinateur portable et le téléphone portable volés.\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Born in the Croatian capital, , Bobek gai

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into French. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The Madhya Police recovered the stolen laptop and mobile phone.\n      hyp1: La police de Madhya Pradesh a récupéré l\'ordinateur portable et le téléphone portable volés.\n      hyp2: La police de Madhya a récupéré l\'ordinateur portable et le téléphone portable volés.\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Using GPS navigation apps can be the easi

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into French. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The Madhya Police recovered the stolen laptop and mobile phone.\n      hyp1: La police de Madhya Pradesh a récupéré l\'ordinateur portable et le téléphone portable volés.\n      hyp2: La police de Madhya a récupéré l\'ordinateur portable et le téléphone portable volés.\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: Flavia Alejandra Gleske Fajin, better kno

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into French. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The Madhya Police recovered the stolen laptop and mobile phone.\n      hyp1: La police de Madhya Pradesh a récupéré l\'ordinateur portable et le téléphone portable volés.\n      hyp2: La police de Madhya a récupéré l\'ordinateur portable et le téléphone portable volés.\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: In the 1930s and 1940s, he was elected fo

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into French. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: The Madhya Police recovered the stolen laptop and mobile phone.\n      hyp1: La police de Madhya Pradesh a récupéré l\'ordinateur portable et le téléphone portable volés.\n      hyp2: La police de Madhya a récupéré l\'ordinateur portable et le téléphone portable volés.\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: In 1924 he was an invited spokesperson fo

In [ ]:
skipped_rows

[0, 34, 50, 59, 72, 79, 82, 88, 93]

## French-English

In [ ]:
trial_ds_fr_en.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,fr-en,"Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.","Mix the two dried powders together, then, with clean and wet hands, press them into a ball.","Mix the dried powders together, then, with clean and wet hands, press them into a ball.",addition,hyp1
1,1,fr-en,Le vote doit encore être ratifié par l'ensemble du CIO lors de sa réunion d'octobre à Copenhague.,The agreement will still have to be ratified by the IOC at its Oct. meeting in Copenhagen.,The agreement will still have to be ratified by the IOC at its May meeting in Copenhagen.,date,hyp2
2,2,fr-en,Le tailleur a appelé la déménageuse.,The tailor called the female mover.,The tailor called the male mover.,gender,hyp2
3,3,fr-en,"Dans les années 1930 et 1940, il fut élu à quatre reprises à travers Al Smith en 1928, et Franklin D. Roosevelt et Harry S. Truman en 1948.","In the 1930s and 1940s, he was elected four times for A Smith in 1928 for Franklin D. Roosevelt and for Harry S. Truman in 1948.","Massachusetts voted for Al Smith in 1928, for Franklin D. Roosevelt four times in the 1930s and 1940s, and for Harry S. Truman in 1948.",named-entity,hyp1
4,4,fr-en,"La 99e a soutenu l'opération Urgent Wury, qui a remplacé le régime stalinien à Grenade peu de temps après son activation.","Shortly after its activation, the 99th supported Operation Urgent Fury, which replaced the Stalinist regime in Grenada.","The 919th supported the Urgent Wury operation, which replaced the Stalinist regime in Grenada shortly after its activation.",number,hyp2


In [ ]:

def detect_hallucination(source, hyp1, hyp2, translation_language):
    # Generate the prompt
    answer_format = {"type": "", "label": ""}

    # prompt for translation generation task
    user_prompt = f'''Given a "src" and two hypotheses "hyp1" and "hyp2", which are translated into {translation_language}. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.
      Please explain your choice.
      Provide the result in the following format: {answer_format}. Here is an example to illustrate this:

      Src: Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.
      hyp1: Mix the two dried powders together, then, with clean and wet hands, press them into a ball.
      hyp2: Mix the dried powders together, then, with clean and wet hands, press them into a ball
      label: hyp1
      type: addition

      Detect which of the following hypotheses is the hallucination.

       Src: {source}
       hyp1 : {hyp1}
       hyp2 : {hyp2}

       Result:
       '''
    #print(user_prompt)

    messages = [
      {"role": "system", "content": "Imagine you are a classifier."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    print(outputs)
    detected_hallu = outputs[0]["generated_text"][len(prompt):]
    print("detected hallucination: ", detected_hallu)

    return detected_hallu

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_fr_en_prompt4.csv"
skipped_rows, df = save_trial_results(data_fr_en, csv_filename, "English")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.\n      hyp1: Mix the two dried powders together, then, with clean and wet hands, press them into a ball.\n      hyp2: Mix the dried powders together, then, with clean and wet hands, press them into a ball\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.\n      hyp1: Mix the two dried powders together, then, with clean and wet hands, press them into a ball.\n      hyp2: Mix the dried powders together, then, with clean and wet hands, press them into a ball\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.\n      hyp1: Mix the two dried powders together, then, with clean and wet hands, press them into a ball.\n      hyp2: Mix the dried powders together, then, with clean and wet hands, press them into a ball\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.\n      hyp1: Mix the two dried powders together, then, with clean and wet hands, press them into a ball.\n      hyp2: Mix the dried powders together, then, with clean and wet hands, press them into a ball\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nImagine you are a classifier.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven a "src" and two hypotheses "hyp1" and "hyp2", which are translated into English. Your task is to detect which of the two hypotheses is hallucinated ("label") and determine what type of hallucination ("type") it is.\n      Please explain your choice.\n      Provide the result in the following format: {\'type\': \'\', \'label\': \'\'}. Here is an example to illustrate this:\n\n      Src: Mélangez les poudres sèches ensemble, puis, avec les mains propres et humides, pressez-les en une boule.\n      hyp1: Mix the two dried powders together, then, with clean and wet hands, press them into a ball.\n      hyp2: Mix the dried powders together, then, with clean and wet hands, press them into a ball\n      label: hyp1\n      type: addition\n\n      Detect which of the following hypotheses is the hallucination.\n\n       Src: 

In [ ]:
skipped_rows

[]

## Convert to comma-separated csv

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_de_en_prompt4_final.csv"


df = pd.read_csv(csv_filename, sep=";")



In [ ]:
df.head()

,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp1


In [ ]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_de_en_prompt4_final.csv", mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_fr_prompt4_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp2


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_fr_en_prompt4_final.csv"


df = pd.read_csv(csv_filename, sep=";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp2


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Detection_task/Llama3/results_llama3_en_de_prompt4_final.csv"


df = pd.read_csv(csv_filename, sep= ";")
df.head()


,id,label
0,0,hyp2
1,1,hyp2
2,2,hyp2
3,3,hyp2
4,4,hyp2


In [ ]:
df.to_csv(csv_filename, mode='w', header=True, index=False)